In [1]:
from pathlib import Path
import sys
import re
import pandas as pd
import shutil
import os
from multiprocessing import Pool
import pickle

from AbPred.call_modeller import ModellerMut
from AbPred.structure_tools import convert_aa, split_mutations


In [2]:
# adapt benchmark dataframe
adapt_dataframe = pd.read_csv('../data/3bdy_adapt.csv',sep=";")
#ab_data_singleMut.sort_values('#Pdb',inplace=True)

PDBS_DIR = Path("../benchmark/adapt_bench/")

adapt_pdb = '../benchmark/adapt_bench/3bdy.protein.pdb'

In [3]:
adapt_dataframe = adapt_dataframe.loc[-adapt_dataframe.Mutation.str.contains('S30b|S30d')]

#  Run mutation protocol by modeller

In [4]:
pdbfile = 'VRC01.pdb'
mutation = 'E54A'
chain = 'G'

# process args
# split mutation string to [wt_aa,aa_number,mut_aa]
mutation_split = split_mutations(mutation)
restype = convert_aa(mutation_split[0])
respos = mutation_split[1]

In [5]:
# run test
mutate1 = ModellerMut(pdbfile, out_dir='out_models')
mutate1.run_mutation(restype=restype, chain=chain, respos=respos, out_tag=".mut")


                         MODELLER 9.20, 2018/05/30, r11208

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2018 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, USA
                   Imperial Cancer Research Fund, London, UK
              Birkbeck College, University of London, London, UK


Kind, OS, HostName, Kernel, Processor: 4, Linux VILION-PC 4.4.0-17134-Microsoft x86_64
Date and time of compi


Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:      2837970    2771.455     2.706

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:      2844396    2777.730     2.713

Dynamically allocated memory at    amaxsequence [B,KiB,MiB]:      2844528    2777.859     2.713

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:      2844825    2778.149     2.713

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:      2854447    2787.546     2.722

Dynamically allocated memory at    amaxsequence [B,KiB,MiB]:      2854647    2787.741     2.722

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:      2855097    2788.181     2.723

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:      2869547    2802.292     2.737

Dynamically allocated memory at    amaxsequence [B,KiB,MiB]:      2869847    2802.585     2.737

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:      2870522    2803.244     2.738

Dynamically allocated memory 

  serious non-bonded atom clash:  1139 1553   2.036
  serious non-bonded atom clash:  1707 3322   2.039
  serious non-bonded atom clash:  1851 2349   2.036
  serious non-bonded atom clash:  2687 3166   2.039
  serious non-bonded atom clash:  3415 3568   2.047
  serious non-bonded atom clash:  3938 4032   2.040
  serious non-bonded atom clash:  4139 4349   2.039
  serious non-bonded atom clash:  4207 4290   2.041
  serious non-bonded atom clash:  4722 4837   2.034
  serious non-bonded atom clash:  5211 5709   2.020
  serious non-bonded atom clash:  5268 5489   2.045

DISTANCE1:  0.00 2.10 2.20 2.30 2.40 2.50 2.60 2.70 2.80 2.90 3.00 3.10 3.20 3.30 3.40
DISTANCE2:  2.10 2.20 2.30 2.40 2.50 2.60 2.70 2.80 2.90 3.00 3.10 3.20 3.30 3.40 3.50
FREQUENCY:    13    0    0    0    0    0    1   17   91  211  287  377  543  714  922


<< end of ENERGY.
randomi_498_> Atoms,selected atoms,random_seed,amplitude:     6089        9        1        4.0000
randomi_496_> Amplitude is > 0; randomization i

In [21]:
def mainModmut(row):
    pdbid = '3bdy'
    partners = 'LH_V'
    mutations = row.Mutation
    pdbname = '{}.protein.pdb'.format(pdbid)
    
    # get argument for mutate function
    modelfile = PDBS_DIR/Path(pdbname)
    mutation_split = split_mutations(mutations)
    # wildtype res
    restyp = convertAAcode(mutation_split[-1])
    respos = mutation_split[1]
    chain = row.Chain
    
    #print(modelfile,restyp,chain,respos)
    #return
    dope_energy =  mutate_model(pdbfile=str(modelfile.resolve()),restyp=restyp,chain=chain,respos=respos,random_seed=10)
    return dope_energy

def mainModwt(row):
    pdbid = '3bdy'
    partners = 'LH_V'
    mutations = row.Mutation
    
    # get argument for mutate function
    mutation_split = split_mutations(mutations)
    # wildtype res
    restyp = convertAAcode(mutation_split[0])
    respos = mutation_split[1]
    chain = row.Chain
    
    pdbname = '{}.protein.{}.mut.pdb'.format(pdbid,chain+mutations[1:])
    modelfile = PDBS_DIR/Path(pdbname)
    dope_energy =  mutate_model(pdbfile=str(modelfile.resolve()),restyp=restyp,chain=chain,respos=respos,random_seed=10)
    return dope_energy

def run_mutation(row,out_tag=".mut"):
    
    # process args    
    pdbid = '3bdy'
    partners = 'LH_V'
    mutation = row.Mutation
    chain = row.Chain
     # split mutation string to [wt_aa,aa_number,mut_aa]
    mutation_split = split_mutations(mutation)
    restype = convert_aa(mutation_split[-1])
    respos = mutation_split[1]
    
    pdbname = '{}.protein.pdb'.format(pdbid)    
    modelfile = PDBS_DIR/Path(pdbname)
    
    # run test
    mutate = ModellerMut(str(modelfile.resolve()), restype=restype, chain=chain, respos=respos, out_dir='out_models')
    mutate.run_mutation(out_tag=out_tag)
    

In [22]:
CWD = Path.cwd()

try:
    #os.chdir('skempiAB-modeller')
    with Pool(3) as p:
        p.map(run_mutation, [row for i,row in adapt_dataframe.iterrows()])
finally:
    os.chdir(CWD)

openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           $(LIB)/restyp.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/resgrp.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL9v20}/modlib/resgrp.lib
rdresgr_266_> Number of residue groups:        2
rdresgr_266_> Number of residue groups:        2
openf___224_> Open           ${MODINSTALL9v20}/modlib/sstruc.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       191566     187.076     0.183openf___224_> Open           ${MODINSTALL9v20}/modlib/sstruc.lib

Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       191566     187.076     0.183
Dynamically allocated memory at   amaxlibraries [B,KiB,MiB]:       191566     187.076     0.183



Dynamically allocated memory a

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       404010     394.541     0.385

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       401010     391.611     0.382

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       399010     389.658     0.381

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       408510     398.936     0.390

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       404010     394.541     0.385

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       401010     391.611     0.382

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       414082     404.377     0.395

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       408510     398.936     0.390

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       404010     394.541     0.385

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       416882     407.111     0.398

Dynamically allocated memory a

openf___224_> Open           ${MODINSTALL9v20}/modlib/mnch2.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/mnch3.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/mnch3.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/mnch3.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/xs4.mat
rdrrwgh_268_> Number of residue types:       21
openf___224_> Open           ${MODINSTALL9v20}/modlib/xs4.mat
openf___224_> Open           $(LIB)/top_heav.lib
rdrrwgh_268_> Number of residue types:       21
read_to_681_> topology.submodel read from topology file:        3
openf___224_> Open           $(LIB)/top_heav.lib
read_to_681_> topology.submodel read from topology file:        3
openf___224_> Open           ${MODINSTALL9v20}/modlib/xs4.mat
rdrrwgh_268_> Number of residue types:       21
openf___224_> Open           $(LIB)/top_heav.lib
read_to_681_> topology.submodel read from topology file:        3
openf___224_> Open           $(LIB)/par.lib

Dynamically allocated me

OSError: pdbnam_____E> Filename for PDB code not found: /mnt/d/Python_projects/Master-thesis/benchmark/adapt_bench/3bdy.protein.pdb
              Directories: 
              Extensions : :.atm:.pdb:.ent:.cif:.crd
              (Also tried prepending 'pdb', looking for .Z, .gz, .bz2, .7z, .xz,
              and trying PDB-style subdirectories - e.g. ab for pdb1abc.ent)


In [11]:
CWD = Path.cwd()

energy_wt = list()
try:
    #os.chdir('skempiAB-modeller')
    with Pool(12) as p:
        dope_energy = p.map(mainModwt, [row for i,row in adapt_dataframe.iterrows()])
        energy_wt.append(dope_energy)
finally:
    os.chdir(CWD)


                         MODELLER 9.20, 2018/05/30, r11208

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2018 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, USA
                   Imperial Cancer Research Fund, London, UK
              Birkbeck College, University of London, London, UK


Kind, OS, HostName, Kernel, Processor: 4, Darwin Mac-Pro-de-Victor.local 14.5.0 x86_64
Date and time of compi




                         MODELLER 9.20, 2018/05/30, r11208

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2018 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York, USA
                      Harvard University, Cambridge, USA
                   Imperial Cancer Research Fund, London, UK
              Birkbeck College, University of London, London, UK


Kind, OS, HostName, Kernel, Processor: 4, Darwin Mac-Pro-de-Victor.local 14.5.0 x86_64
Date and time of com

openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/radii14.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/af_mnchdef.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/af_mnchdef.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/af_mnchdef.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/af_mnchdef.lib
openf___224_> Open           ${MODINSTALL9v20}/modlib/af_mnchdef

read_to_681_> topology.submodel read from topology file:        3
read_to_681_> topology.submodel read from topology file:        3
openf___224_> Open           $(LIB)/top_heav.lib
read_to_681_> topology.submodel read from topology file:        3
openf___224_> Open           $(LIB)/par.lib
openf___224_> Open           $(LIB)/par.lib
openf___224_> Open           $(LIB)/par.lib
openf___224_> Open           $(LIB)/par.lib
openf___224_> Open           $(LIB)/par.lib
openf___224_> Open           $(LIB)/par.lib
openf___224_> Open           $(LIB)/par.lib
openf___224_> Open           $(LIB)/par.lib
openf___224_> Open           $(LIB)/par.lib
openf___224_> Open           $(LIB)/par.lib

Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       393430     384.209     0.375
Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       393430     384.209     0.375
Dynamically allocated memory at  amaxparameters [B,KiB,MiB]:       393430     384.209     0.375
Dynamically allocated me


read_pa_232_> parameters    BONDS   ANGLS  DIHEDS IMPROPS    MODE
openf___224_> Open           /Users/vfica/Documents/GitHub/Master-thesis/benchmark/adapt_bench/3bdy.protein.H98M.mut.pdb
openf___224_> Open           /Users/vfica/Documents/GitHub/Master-thesis/benchmark/adapt_bench/3bdy.protein.H98S.mut.pdb

Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:       442527     432.155     0.422openf___224_> Open           /Users/vfica/Documents/GitHub/Master-thesis/benchmark/adapt_bench/3bdy.protein.H98W.mut.pdb
openf___224_> Open           /Users/vfica/Documents/GitHub/Master-thesis/benchmark/adapt_bench/3bdy.protein.H98Q.mut.pdb
                              227     561     661     112       0
openf___224_> Open           /Users/vfica/Documents/GitHub/Master-thesis/benchmark/adapt_bench/3bdy.protein.H98T.mut.pdb
read_pa_232_> parameters    BONDS   ANGLS  DIHEDS IMPROPS    MODE


Dynamically allocated memory at amaxcoordinates [B,KiB,MiB]:       442527     432.155     0.422
Dy

...

  serious non-bonded atom clash:  1786 2329   2.070
11 Mainchain Phi dihedral restraints  :       0       0      0   0.000   0.000      0.0000       1.000
12 Mainchain Psi dihedral restraints  :       0       0      0   0.000   0.000      0.0000       1.000
>> Normalized DOPE z score: -0.913
13 Mainchain Omega dihedral restraints:       2       0      0   3.117   3.117     0.22926       1.000
openf___224_> Open           /Users/vfica/Documents/GitHub/Master-thesis/benchmark/adapt_bench/3bdy.protein.H53R.mut.H53T.wt.pdb
14 Sidechain Chi_1 dihedral restraints:       1       0      0  18.146  18.146     0.74845       1.000
15 Sidechain Chi_2 dihedral restraints:       1       0      0  17.639  17.639     0.55094       1.000
16 Sidechain Chi_3 dihedral restraints:       0       0      0   0.000   0.000      0.0000       1.000
iatmcls_286W> MODEL atom not classified:  GLU:OXT  GLU
17 Sidechain Chi_4 dihedral restraints:       0       0      0   0.000   0.000      0.0000       1.000
18 Disul


DISTANCE1:  0.00 2.10 2.20 2.30 2.40 2.50 2.60 2.70 2.80 2.90 3.00 3.10 3.20 3.30 3.40Number of all static, selected restraints         :    20121      80

COVALENT_CYS                                      :        F
DISTANCE2:  2.10 2.20 2.30 2.40 2.50 2.60 2.70 2.80 2.90 3.00 3.10 3.20 3.30 3.40 3.50
NONBONDED_SEL_ATOMS                               :        1
FREQUENCY:     7    0    0    0    4   11   17   70  124  164  218  300  388  474  614
Number of non-bonded pairs (excluding 1-2,1-3,1-4):   768847

Dynamic pairs routine                             : 1, NATM x NATM double loop

<< end of ENERGY.Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390

LENNARD_JONES_SWITCH                              :    6.500   7.500
COULOMB_JONES_SWITCH                              :    6.500   7.500
>> Model assessment by DOPE potential
RESIDUE_SPAN_RANGE                                :        1    9999
NLOGN_USE                                         :       15
CONTACT_SHELL       

 9 Distance restraints 1 (CA-CA)      :       0       0      0   0.000   0.000      0.0000       1.000
10 Distance restraints 2 (N-O)        :       0       0      0   0.000   0.000      0.0000       1.000
11 Mainchain Phi dihedral restraints  :       0       0      0   0.000   0.000      0.0000       1.000
12 Mainchain Psi dihedral restraints  :       0       0      0   0.000   0.000      0.0000       1.000
13 Mainchain Omega dihedral restraints:       2       0      0   4.366   4.366     0.44987       1.000
14 Sidechain Chi_1 dihedral restraints:       1       0      0   6.388   6.388    -0.35023       1.000
15 Sidechain Chi_2 dihedral restraints:       1       0      0 167.294 167.294     0.17112       1.000
16 Sidechain Chi_3 dihedral restraints:       0       0      0   0.000   0.000      0.0000       1.000
17 Sidechain Chi_4 dihedral restraints:       0       0      0   0.000   0.000      0.0000       1.000
18 Disulfide distance restraints      :       0       0      0   0.000   

COULOMB_JONES_SWITCH                              :    6.500   7.500
RESIDUE_SPAN_RANGE                                :        0   99999
NLOGN_USE                                         :       15
CONTACT_SHELL                                     :    4.000
DYNAMIC_PAIRS,_SPHERE,_COULOMB,_LENNARD,_MODELLER :        T       F       F       T       F
SPHERE_STDV                                       :    0.050
RADII_FACTOR                                      :    0.820
Current energy                                    :          19.5360





Summary of the restraint violations: 

   NUM     ... number of restraints.
   NUMVI   ... number of restraints with RVIOL > VIOL_REPORT_CUT[i].
   RVIOL   ... relative difference from the best value.
   NUMVP   ... number of restraints with -Ln(pdf) > VIOL_REPORT_CUT2[i].
   RMS_1   ... RMS(feature, minimally_violated_basis_restraint, NUMB).
   RMS_2   ... RMS(feature, best_value, NUMB).
   MOL.PDF ... scaled contribution to -Ln(Molecular pdf).



Number of all static, selected restraints         :    20121      51
COVALENT_CYS                                      :        F
NONBONDED_SEL_ATOMS                               :        1
Number of non-bonded pairs (excluding 1-2,1-3,1-4):   768860
Dynamic pairs routine                             : 1, NATM x NATM double loop
Atomic shift for contacts update (UPDATE_DYNAMIC) :    0.390
LENNARD_JONES_SWITCH                              :    6.500   7.500
COULOMB_JONES_SWITCH                              :    6.500   7.500
RESIDUE_SPAN_RANGE                                :        1    9999
NLOGN_USE                                         :       15
CONTACT_SHELL                                     :   15.000
DYNAMIC_PAIRS,_SPHERE,_COULOMB,_LENNARD,_MODELLER :        T       F       F       F       T
SPHERE_STDV                                       :    0.050
RADII_FACTOR                                      :    0.820
Current energy                                    :      -56451.

In [10]:
import concurrent
import multiprocessing

In [15]:
import functools
from concurrent import futures
import os

In [16]:
functools.partial?

In [ ]:
starter = partial(runencoder, vq=args.videoquality,
                  aq=args.audioquality)
errmsg = 'conversion of {} failed, return code {}'
with concurrent.futures.ProcessPoolExecutor(max_workers=os.cpu_count()) as tp:
    fl = [tp.submit(starter, t) for t in args.files]
    for fut in concurrent.futures.as_completed(fl):
        fn, rv = fut.result()
        if rv == 0:
            logging.info('finished "{}"'.format(fn))
        elif rv < 0:
            ls = 'file "{}" has unknown extension, ignoring it.'
            logging.warning(ls.format(fname))
        else:
            logging.error(errmsg.format(fn, rv))